<a href="https://colab.research.google.com/github/Pengyu-gis/Historical-Aerial-Photos/blob/main/Aerial_Images_Colorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **<font color='blue'> Artistic Colorizer </font>**

#◢ DeOldify - Colorize your own photos!

####**Credits:**

Special thanks to:

Matt Robinson and María Benavente for pioneering the DeOldify image colab notebook.  

Dana Kelley for doing things, breaking stuff & having an opinion on everything.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU


#◢ Git clone and install DeOldify

In [1]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2615, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 2615 (delta 91), reused 204 (delta 71), pack-reused 2346 (from 1)
Receiving objects: 100% (2615/2615), 69.71 MiB | 11.28 MiB/s, done.
Resolving deltas: 100% (1174/1174), done.


In [2]:
!cd DeOldify

/content/DeOldify


#◢ Setup

In [3]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

In [ ]:
!pip install -r requirements-colab.txt

In [5]:
import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.


In [6]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

mkdir: cannot create directory ‘models’: File exists
--2024-10-06 20:28:48--  https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth
Resolving data.deepai.org (data.deepai.org)... 169.150.249.167, 2400:52e0:1a01::1108:1
Connecting to data.deepai.org (data.deepai.org)|169.150.249.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255144681 (243M) [application/octet-stream]
Saving to: ‘./models/ColorizeArtistic_gen.pth’

./models/ColorizeAr 100%[===================>] 243.32M  4.97MB/s    in 49s     

2024-10-06 20:29:38 (4.92 MB/s) - ‘./models/ColorizeArtistic_gen.pth’ saved [255144681/255144681]



In [7]:
colorizer = get_image_colorizer(artistic=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can als

#◢ Instructions

### source_url
Type in a url to a direct link of an image.  Usually that means they'll end in .png, .jpg, etc. NOTE: If you want to use your own image, upload it first to a site like Imgur.

### render_factor
The default value of 35 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the image is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality images in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality images, but the colors may get slightly washed out.

### watermarked
Selected by default, this places a watermark icon of a palette at the bottom left corner of the image.  This is intended to be a standard way to convey to others viewing the image that it is colorized by AI. We want to help promote this as a standard, especially as the technology continues to improve and the distinction between real and fake becomes harder to discern. This palette watermark practice was initiated and lead by the company MyHeritage in the MyHeritage In Color feature (which uses a newer version of DeOldify than what you're using here).

#### How to Download a Copy
Simply right click on the displayed image and click "Save image as..."!

## Pro Tips

You can evaluate how well the image is rendered at each render_factor by using the code at the bottom (that cell under "See how well render_factor values perform on a frame here").

#◢ Colorize!!

In [11]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 82.8 MB/s eta 0:00:00


In [35]:
import rasterio as rio
import numpy as np
from rasterio.plot import reshape_as_image, reshape_as_raster
from PIL import Image

# Load the TIFF file using rasterio
tiff_input_path = '/content/Clip1_0087_x_24.tif'
with rio.open(tiff_input_path) as src:
    meta = src.meta.copy()  # Copy the metadata (including CRS)
    image_array = src.read()  # Read the image as a numpy array

    # Convert the image to RGB (assuming the first 3 bands correspond to RGB channels)
    if image_array.shape[0] > 3:
        image_array_rgb = image_array[:3, :, :]  # Use only the first three bands
    else:
        image_array_rgb = image_array  # If already 3-channel

    # Reshape as image for processing (channels-last format)
    image_rgb = reshape_as_image(image_array_rgb)

    # Use DeOldify to colorize the image
    # Convert the numpy array to a PIL Image
    image_rgb = Image.fromarray(image_rgb)

    # Save the image
    image_rgb.save('temp.png')
    #The get_transformed_image function expects a path not a PIL Image
    # Pass the path to the saved image to colorizer.get_transformed_image
    colorized_image = colorizer.get_transformed_image('temp.png', render_factor=35, watermarked=True)

    # Convert the colorized image back to raster format (channels-first)
    colorized_image_array = np.array(colorized_image)

    # Ensure that the array is in (bands, height, width) format
    colorized_image_raster = colorized_image_array.transpose(2, 0, 1)

# Update metadata for the output TIFF (ensure 3 bands)
meta.update({
    "count": 3,  # Number of bands (RGB)
    "dtype": 'uint8',  # Data type should match the output image type
    "height": colorized_image_raster.shape[1],  # Update height to match the colorized image
    "width": colorized_image_raster.shape[2]  # Update width to match the colorized image
})

# Save the colorized TIFF while preserving CRS and metadata
tiff_output_path = 'output_image_test1.tif'
with rio.open(tiff_output_path, 'w', **meta) as dst:
    dst.write(colorized_image_raster)

print(f"Colorized image saved at {tiff_output_path}")


Colorized image saved at output_image_test1.tif


---
#⚙ Recommended image sources
* [/r/TheWayWeWere](https://www.reddit.com/r/TheWayWeWere/)